In [1]:
# only for taurus
import os
print(os.getcwd())
os.chdir("notebooks")
print(os.getcwd())

/data/horse/ws/jori152b-medinf/KP_MedInf
/data/horse/ws/jori152b-medinf/KP_MedInf/notebooks


In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt
import seaborn as sns

In [7]:
# File paths and separator
DATA_PATH_stages = "../data/extracted/kdigo_stages_measured.csv"
DATA_PATH_labs = "../data/extracted/labs_original.csv"
DATA_PATH_labs_extended = "../data/extracted/labs_extended.csv"
DATA_PATH_labs_new = "../data/extracted/labs_new.csv"
# DATA_PATH_vitals = "../data/extracted/vitals-kdigo_stages_measured.csv"
DATA_PATH_vitals = "../data/extracted/vitals.csv"
# DATA_PATH_vents = "../data/extracted/vents-vasopressor-sedatives-kdigo_stages_measured.csv"
DATA_PATH_vents = "../data/extracted/vents_vasopressor_sedatives.csv"
# DATA_PATH_detail = "../data/extracted/icustay_detail-kdigo_stages_measured.csv"
DATA_PATH_detail = "../data/extracted/icustay_detail.csv"
DATA_PATH_heightweight = "../data/extracted/heightweight.csv"
DATA_PATH_calcium = "../data/extracted/calcium.csv"
DATA_PATH_inr_max = "../data/extracted/inr_max.csv"
SEPARATOR = ";"

# Constants
IMPUTE_EACH_ID = False
IMPUTE_COLUMN = False
TESTING = False
TEST_SIZE = 0.05
SPLIT_SIZE = 0.2
MAX_DAYS = 35
CLASS1 = True
ALL_STAGES = False
MAX_FEATURE_SET = True
FIRST_TURN_POS = True
TIME_SAMPLING = True
SAMPLING_INTERVAL = '6H'
RESAMPLE_LIMIT = 16
MOST_COMMON = False
IMPUTE_METHOD = 'most_frequent'
FILL_VALUE = 0
ADULTS_MIN_AGE = 18
ADULTS_MAX_AGE = 120
NORMALIZATION = 'min-max'
HOURS_AHEAD = 48
NORM_TYPE = 'min_max'
RANDOM = 42

def filter_by_length_of_stay(X):
    drop_list = []
    long_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) > MAX_DAYS)

    for icustay_id, is_long in long_stays.items():
        if is_long:
            max_time = X[X['icustay_id'] == icustay_id]['charttime'].max() - pd.to_timedelta(MAX_DAYS, unit='D')
            X = X[~((X['icustay_id'] == icustay_id) & (X['charttime'] < max_time))]

    short_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) < (HOURS_AHEAD/24))
    drop_list = short_stays[short_stays].index.tolist()

    X = X[~X.icustay_id.isin(drop_list)]
    return X

In [12]:
# Load datasets
print("Loading datasets...")
X = pd.read_csv(DATA_PATH_stages, sep=SEPARATOR)
X.drop(["aki_stage_creat", "aki_stage_uo"], axis=1, inplace=True)
X = X.dropna(how='all', subset=['creat', 'uo_rt_6hr', 'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage'])
X['charttime'] = pd.to_datetime(X['charttime'])

print(len(X))
print(X['aki_stage'].value_counts())

dataset_detail = pd.read_csv(DATA_PATH_detail, sep=SEPARATOR)
dataset_detail.drop(['dod', 'admittime', 'dischtime', 'los_hospital', 'ethnicity', 
                     'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime', 
                     'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay'], axis=1, inplace=True)

dataset_labs = pd.read_csv(DATA_PATH_labs, sep=SEPARATOR)
dataset_labs = dataset_labs.dropna(subset=['charttime']).dropna(subset=dataset_labs.columns[4:], how='all')
dataset_labs['charttime'] = pd.to_datetime(dataset_labs['charttime'])
dataset_labs = dataset_labs.sort_values(by=['icustay_id', 'charttime'])
dataset_labs.drop(['albumin_min', 'albumin_max','bilirubin_min', 'bilirubin_max','bands_min', 'bands_max',
                   'lactate_min', 'lactate_max','platelet_min', 'platelet_max','ptt_min', 'ptt_max', 
                   'inr_min', 'inr_max', 'pt_min', 'pt_max'], axis = 1, inplace = True)

# dataset_labs_new = pd.read_csv(DATA_PATH_labs_new, sep=SEPARATOR)
# dataset_labs_new = dataset_labs_new.dropna(subset=['charttime']).dropna(subset=dataset_labs_new.columns[4:], how='all')
# dataset_labs_new['charttime'] = pd.to_datetime(dataset_labs_new['charttime'])
# dataset_labs_new = dataset_labs_new.sort_values(by=['icustay_id', 'charttime'])


# dataset_labs_extended = pd.read_csv(DATA_PATH_labs_extended, sep=SEPARATOR)
# dataset_labs_extended = dataset_labs_extended.dropna(subset=['charttime']).dropna(subset=dataset_labs_extended.columns[4:], how='all')
# dataset_labs_extended['charttime'] = pd.to_datetime(dataset_labs_extended['charttime'])
# dataset_labs_extended = dataset_labs_extended.sort_values(by=['icustay_id', 'charttime'])

dataset_vitals = pd.read_csv(DATA_PATH_vitals, sep=SEPARATOR)
dataset_vents = pd.read_csv(DATA_PATH_vents, sep=SEPARATOR)
dataset_vitals.drop(["heartrate_min", "heartrate_max", "sysbp_min", "sysbp_max", "diasbp_min", "diasbp_max",
                        'meanbp_min', 'meanbp_max', 'tempc_min', 'tempc_max', "resprate_min", "resprate_max", 
                        "spo2_min", "spo2_max", "glucose_min", "glucose_max"], axis=1, inplace=True)
dataset_vitals['charttime'] = pd.to_datetime(dataset_vitals['charttime'])
dataset_vents['charttime'] = pd.to_datetime(dataset_vents['charttime'])
dataset_vitals = dataset_vitals.dropna(subset=dataset_vitals.columns[4:], how='all')
dataset_vitals = dataset_vitals.sort_values(by=['icustay_id', 'charttime'])
dataset_vents = dataset_vents.sort_values(by=['icustay_id', 'charttime'])

dataset_heightweight = pd.read_csv(DATA_PATH_heightweight, sep=SEPARATOR)
dataset_heightweight = dataset_heightweight.dropna(subset=['icustay_id', 'height_first', 'weight_first'], how='all')
dataset_heightweight = dataset_heightweight.sort_values(by=['icustay_id'])

dataset_calcium = pd.read_csv(DATA_PATH_calcium, sep=SEPARATOR)
dataset_calcium.drop(["hadm_id"], axis=1, inplace=True)
dataset_calcium['charttime'] = pd.to_datetime(dataset_calcium['charttime'])
dataset_calcium = dataset_calcium.sort_values(by=['icustay_id', 'charttime'])

dataset_inr_max = pd.read_csv(DATA_PATH_inr_max, sep=SEPARATOR)
dataset_inr_max.drop(["hadm_id", "subject_id"], axis=1, inplace=True)
dataset_inr_max = dataset_inr_max.sort_values(by=['icustay_id'])

# Calculate mean for each pair and drop original columns
column_pairs = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
                ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
                ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
                ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
                ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
                ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
                ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
                ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
                ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max')]

for min_col, max_col in column_pairs:
    try:
        mean_col = min_col.rsplit('_', 1)[0] + '_mean'
        dataset_labs[mean_col] = dataset_labs[[min_col, max_col]].mean(axis=1)
        dataset_labs.drop([min_col, max_col], axis=1, inplace=True)
    except:
        pass

# column_pairs_new = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
#                 ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
#                 ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
#                 ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
#                 ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
#                 ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
#                 ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
#                 ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
#                 ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max'), ('bilirubin_total_min', 'bilirubin_total_max'), 
#                 ('estimated_gfr_min', 'estimated_gfr_max'), ('phosphate_min', 'phosphate_max'), 
#                 ('urea_nitrogen_min', 'urea_nitrogen_max'), ('uric_acid_min', 'uric_acid_max'), 
#                 ('calcium_total_min', 'calcium_total_max'), ('inr_pt_min', 'inr_pt_max'), 
#                 ('platelet_count_min', 'platelet_count_max')]

# for min_col, max_col in column_pairs_new:
#     mean_col = min_col.rsplit('_', 1)[0] + '_mean'
#     dataset_labs_new[mean_col] = dataset_labs_new[[min_col, max_col]].mean(axis=1)
#     dataset_labs_new.drop([min_col, max_col], axis=1, inplace=True)

# column_pairs_extended = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
#                 ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
#                 ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
#                 ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
#                 ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
#                 ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
#                 ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
#                 ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
#                 ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max'), 
#                 ('gfr_min', 'gfr_max'), ('phosphate_min', 'phosphate_max'),('uric_acid_min', 'uric_acid_max'), 
#                 ('calcium_min', 'calcium_max')]


# for min_col, max_col in column_pairs_extended:
#     mean_col = min_col.rsplit('_', 1)[0] + '_mean'
#     dataset_labs_extended[mean_col] = dataset_labs_extended[[min_col, max_col]].mean(axis=1)
#     dataset_labs_extended.drop([min_col, max_col], axis=1, inplace=True)
    
# dataset_labs_extended.drop(['gfr_mean'], axis=1, inplace=True)

# Merge datasets
if MAX_FEATURE_SET:
    X = X.merge(dataset_labs, on=["icustay_id", "charttime"], how="outer")
    # X = X.merge(dataset_labs_new, on=["icustay_id", "charttime"], how="outer")
    # X = X.merge(dataset_labs_extended, on=["icustay_id", "charttime"], how="outer")
    X = X.merge(dataset_vitals, on=["icustay_id", "charttime", "subject_id", "hadm_id"], how="outer")
    X = X.merge(dataset_vents, on=["icustay_id", "charttime"], how="outer")
    X.drop(["subject_id"], axis=1, inplace=True)
    X = X.merge(dataset_calcium, on=["icustay_id", "charttime"], how="outer")

Loading datasets...
3737147
aki_stage
0    3000795
2     353266
3     207759
1     175327
Name: count, dtype: int64


In [16]:
dataset_vitals.describe()

,subject_id,hadm_id,icustay_id,charttime,heartrate_mean,sysbp_mean,diasbp_mean,meanbp_mean,resprate_mean,tempc_mean,spo2_mean,glucose_mean
count,9.067143e+06,9.067143e+06,9.067143e+06,9067143,7.859472e+06,5.776911e+06,5.775124e+06,5.792180e+06,6.271224e+06,1.733080e+06,6.078894e+06,1.261644e+06
mean,3.323994e+04,1.498799e+05,2.506103e+05,2152-04-21 23:31:49.216401408,1.010353e+02,1.215073e+02,6.027826e+01,7.914960e+01,2.020792e+01,3.700486e+01,9.708057e+01,1.425699e+02
min,3.000000e+00,1.000010e+05,2.000010e+05,2100-06-08 00:18:00,1.460000e-01,6.000000e-02,3.200000e-01,4.300000e-01,1.700000e-01,1.500000e+01,5.000000e-01,1.060000e-01
25%,1.172200e+04,1.253330e+05,2.258250e+05,2127-12-28 14:00:00,7.700000e+01,1.040000e+02,5.000000e+01,6.800000e+01,1.600000e+01,3.644444e+01,9.600000e+01,1.070000e+02
50%,2.369500e+04,1.492370e+05,2.509010e+05,2153-01-22 13:15:00,9.200000e+01,1.190000e+02,5.900000e+01,7.700000e+01,2.000000e+01,3.700000e+01,9.800000e+01,1.290000e+02
75%,5.311900e+04,1.750200e+05,2.754880e+05,2177-08-03 22:00:00,1.170000e+02,1.370000e+02,6.900000e+01,8.866670e+01,2.400000e+01,3.755555e+01,9.900000e+01,1.600000e+02
max,9.999900e+04,1.999990e+05,2.999990e+05,2209-08-07 16:00:00,2.920000e+02,3.550000e+02,2.980000e+02,2.990000e+02,6.900000e+01,4.650000e+01,1.000000e+02,9.999990e+05
std,2.792436e+04,2.876894e+04,2.887192e+04,NaN,3.265558e+01,2.368750e+01,1.455439e+01,1.673071e+01,6.043288e+00,8.432392e-01,3.493982e+00,1.260194e+03


In [17]:
pd.set_option('display.max_columns', None)
X.describe()    

,icustay_id,charttime,creat,uo_rt_6hr,uo_rt_12hr,uo_rt_24hr,aki_stage,hadm_id,aniongap_mean,bicarbonate_mean,creatinine_mean,chloride_mean,glucose_mean_x,hematocrit_mean,hemoglobin_mean,potassium_mean,sodium_mean,bun_mean,wbc_mean,heartrate_mean,sysbp_mean,diasbp_mean,meanbp_mean,resprate_mean,tempc_mean,spo2_mean,glucose_mean_y,vent,vasopressor,sedative,subject_id,calcium
count,1.360420e+07,13589169,386396.000000,3.267035e+06,3.267035e+06,3.267035e+06,3.737147e+06,1.018621e+07,727962.000000,739479.000000,741939.000000,807392.000000,936984.000000,921519.000000,782635.000000,1.012462e+06,845254.000000,739446.000000,687683.000000,7.859472e+06,5.776911e+06,5.775124e+06,5.792180e+06,6.271224e+06,1.733080e+06,6.078894e+06,1.261645e+06,1.352810e+07,1.352810e+07,1.352810e+07,239028.000000,91775.000000
mean,2.503809e+05,2152-01-27 20:04:09.496556544,1.524148,1.643963e+00,1.491771e+00,1.438268e+00,4.027500e-01,1.498837e+05,13.746018,25.259717,1.542183,103.824401,134.087883,30.411020,10.267063,4.138491e+00,138.393092,30.991901,11.251299,1.010353e+02,1.215073e+02,6.027826e+01,7.914960e+01,2.020792e+01,3.700486e+01,9.708057e+01,1.425699e+02,4.075521e-01,1.529394e-01,2.183027e-01,33145.886348,1.135195
min,2.000010e+05,2100-06-07 09:20:00,0.000000,-6.637170e+01,-6.637170e+01,-4.694840e+01,0.000000e+00,1.000010e+05,1.000000,2.000000,0.050000,1.600000,0.139000,2.000000,1.200000,6.000000e-01,1.070000,1.000000,0.100000,1.460000e-01,6.000000e-02,3.200000e-01,4.300000e-01,1.700000e-01,1.500000e+01,5.000000e-01,1.060000e-01,0.000000e+00,0.000000e+00,0.000000e+00,3.000000,0.090000
25%,2.255360e+05,2127-07-12 19:04:00,0.700000,5.960000e-01,6.250000e-01,6.612000e-01,0.000000e+00,1.252850e+05,11.000000,22.000000,0.700000,100.000000,101.000000,26.900000,9.000000,3.700000e+00,135.000000,15.000000,7.000000,7.700000e+01,1.040000e+02,5.000000e+01,6.800000e+01,1.600000e+01,3.644444e+01,9.600000e+01,1.070000e+02,0.000000e+00,0.000000e+00,0.000000e+00,11789.000000,1.070000
50%,2.506780e+05,2152-07-28 18:00:00,1.000000,1.059200e+00,1.071700e+00,1.091300e+00,0.000000e+00,1.492560e+05,13.000000,25.000000,1.000000,104.000000,121.000000,29.800000,10.100000,4.100000e+00,138.000000,23.000000,9.900000,9.200000e+01,1.190000e+02,5.900000e+01,7.700000e+01,2.000000e+01,3.700000e+01,9.800000e+01,1.290000e+02,0.000000e+00,0.000000e+00,0.000000e+00,23657.000000,1.130000
75%,2.752700e+05,2177-04-29 07:26:00,1.600000,1.882600e+00,1.783300e+00,1.729200e+00,0.000000e+00,1.750230e+05,16.000000,28.000000,1.700000,108.000000,150.000000,33.300000,11.300000,4.400000e+00,141.000000,40.000000,13.700000,1.170000e+02,1.370000e+02,6.900000e+01,8.866670e+01,2.400000e+01,3.755555e+01,9.900000e+01,1.600000e+02,1.000000e+00,0.000000e+00,0.000000e+00,52641.000000,1.190000
max,2.999990e+05,2210-08-24 05:53:00,138.000000,9.000000e+02,9.000000e+02,9.000000e+02,3.000000e+00,1.999990e+05,118.000000,65.000000,138.000000,198.000000,3565.000000,77.700000,43.000000,2.750000e+01,184.000000,290.000000,846.700000,2.920000e+02,3.550000e+02,2.980000e+02,2.990000e+02,6.900000e+01,4.650000e+01,1.000000e+02,9.999990e+05,1.000000e+00,1.000000e+00,1.000000e+00,99999.000000,96.000000
std,2.886292e+04,NaN,1.620051,3.591551e+00,3.016078e+00,2.875846e+00,8.735885e-01,2.878383e+04,3.876696,5.077160,1.579590,6.351784,65.007258,5.317715,1.863787,6.698996e-01,5.206882,24.166935,9.247759,3.265558e+01,2.368750e+01,1.455439e+01,1.673071e+01,6.043288e+00,8.432392e-01,3.493982e+00,1.260193e+03,4.913791e-01,3.599291e-01,4.130940e-01,27859.399825,0.555198


In [5]:
# take only head of X
X = X.head(50000)

In [18]:
print("Filtering patients by age and length of stay...")
# Filtering patients by age and length of stay
dataset_detail = dataset_detail[dataset_detail['admission_age'] >= ADULTS_MIN_AGE]
adults_icustay_id_list = dataset_detail['icustay_id'].unique()
X = X[X.icustay_id.isin(adults_icustay_id_list)].sort_values(by=['icustay_id', 'charttime'])

X = filter_by_length_of_stay(X)
dataset_detail = dataset_detail[dataset_detail.icustay_id.isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])

Filtering patients by age and length of stay...


/tmp/ipykernel_219820/53447784.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  long_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) > MAX_DAYS)


In [17]:
label = ['aki_stage']
skip = ['icustay_id', 'charttime', 'aki_stage']
discrete_feat = ['sedative', 'vasopressor', 'vent', 'hadm_id']
skip.extend(discrete_feat)    
numeric_feat = list(X.columns.difference(skip))

In [10]:
print(os.getcwd())


/data/horse/ws/jori152b-medinf/KP_MedInf/notebooks


In [13]:
# save to csv
X.to_csv('../data/analysis/data_preprocessed_extended__filtered_before_resampling.csv', index=False)

In [20]:
X = pd.read_csv('../data/analysis/data_preprocessed_extended__filtered_before_resampling.csv')

In [21]:
X['charttime'] = pd.to_datetime(X['charttime'])


In [24]:
X_old = X.copy()

In [42]:
X = X_old.copy()

In [14]:
SAMPLING_INTERVALS = ['1H', '2H', '3H', '4H', '6H']


In [ ]:
dataset_detail = dataset_detail[dataset_detail['icustay_id'].isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])
dataset_detail = pd.get_dummies(dataset_detail, columns=['gender', 'ethnicity_grouped'])
dataset_detail.drop(['subject_id', 'hadm_id'], axis=1, inplace=True)

In [24]:
# saving every sampling interval
# Resampling
for SAMPLING_INTERVAL in SAMPLING_INTERVALS:
    if TIME_SAMPLING:
        
        # Set index and group by 'icustay_id' before resampling
        X_resampled = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
        
        # Resample and aggregate features
        if MAX_FEATURE_SET:
            X_discrete = X_resampled[discrete_feat].max().fillna(FILL_VALUE).astype(np.int64)
        X_numeric = X_resampled[numeric_feat].mean()
        X_label = X_resampled['aki_stage'].max()

        print("Merging sampled features")
        try:
            X_resampled = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
        except:
            X_resampled = pd.concat([X_numeric, X_label], axis=1).reset_index()



    # Forward fill again after resampling
    X_resampled['aki_stage'] = X_resampled.groupby('icustay_id')['aki_stage'].ffill(limit=RESAMPLE_LIMIT).fillna(0)



    # Ensure binary values (convert any positive number to 1)
    X_resampled['aki_stage'] = (X_resampled['aki_stage'] > 0).astype(int)

    # Shifting labels
    shift_steps = HOURS_AHEAD // int(SAMPLING_INTERVAL[:-1])
    X_resampled['aki_stage'] = X_resampled.groupby('icustay_id')['aki_stage'].shift(-shift_steps)
    X_resampled = X_resampled.dropna(subset=['aki_stage'])

    # Merging not time-dependent data

    X_resampled = X_resampled.merge(dataset_detail, on='icustay_id')
    X_resampled = X_resampled.merge(dataset_heightweight, on='icustay_id')
    X_resampled = X_resampled.merge(dataset_inr_max, on='icustay_id')

    # If no imputation method selected or only impute each id, for the remaining nan impute direclty with FILL_VALUE
    X_resampled = X_resampled.fillna(FILL_VALUE) 


    # Save preprocessed data
    X_resampled.to_csv(f'../data/preprocessed/preprocessed_data_extended_{SAMPLING_INTERVAL}.csv', index=False)




/tmp/ipykernel_834289/3017786234.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X_resampled = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


In [41]:
# Resampling
if TIME_SAMPLING:
    
    # Set index and group by 'icustay_id' before resampling
    X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
    
    # Resample and aggregate features
    if MAX_FEATURE_SET:
        X_discrete = X[discrete_feat].max()
    X_numeric = X[numeric_feat].mean()
    X_label = X['aki_stage'].max()

    print("Merging sampled features")
    try:
        X = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
    except:
        X = pd.concat([X_numeric, X_label], axis=1).reset_index()
        




C:\Users\derqu\AppData\Local\Temp\ipykernel_8124\3895676415.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


KeyboardInterrupt: 

In [ ]:
# Resampling
if TIME_SAMPLING:
    
    # Set index and group by 'icustay_id' before resampling
    X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
    
    # Resample and aggregate features
    if MAX_FEATURE_SET:
        X_discrete = X[discrete_feat].max().fillna(FILL_VALUE).astype(np.int64)
    X_numeric = X[numeric_feat].mean()
    X_label = X['aki_stage'].max()

    print("Merging sampled features")
    try:
        X = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
    except:
        X = pd.concat([X_numeric, X_label], axis=1).reset_index()



# Forward fill again after resampling
X['aki_stage'] = X.groupby('icustay_id')['aki_stage'].ffill(limit=RESAMPLE_LIMIT).fillna(0)



# Ensure binary values (convert any positive number to 1)
X['aki_stage'] = (X['aki_stage'] > 0).astype(int)

# Shifting labels
shift_steps = HOURS_AHEAD // int(SAMPLING_INTERVAL[:-1])
X['aki_stage'] = X.groupby('icustay_id')['aki_stage'].shift(-shift_steps)
X = X.dropna(subset=['aki_stage'])

# Merging not time-dependent data
dataset_detail = dataset_detail[dataset_detail['icustay_id'].isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])
dataset_detail = pd.get_dummies(dataset_detail, columns=['gender', 'ethnicity_grouped'])
dataset_detail.drop(['subject_id', 'hadm_id'], axis=1, inplace=True)
X = X.merge(dataset_detail, on='icustay_id')
X = X.merge(dataset_heightweight, on='icustay_id')
X = X.merge(dataset_inr_max, on='icustay_id')

# If no imputation method selected or only impute each id, for the remaining nan impute direclty with FILL_VALUE
X = X.fillna(FILL_VALUE) 


C:\Users\derqu\AppData\Local\Temp\ipykernel_8124\2742523973.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


Merging sampled features


In [15]:

# Save preprocessed data
X.to_csv('../data/preprocessed/preprocessed_data_extended_small.csv', index=False)